## 可视化工具

wandb / tensorboard / visdom

### wamdn

```shell
pip install wandb
```

```python
import wandb
wandb.init(project='project_name', entity='your_wandb_username')

# 在训练循环中记录指标
wandb.log({'loss': loss_value, 'accuracy': accuracy_value})

# 可视化模型
wandb.watch(model)

# 保存模型
wandb.save('model.pth')
```

多多参考官网文档：https://docs.wandb.ai/

### tensorboard

```
### tensorboard

```shell
pip install tensorboard
```

```python
from torch.utils.tensorboard import SummaryWriter
import numpy as np

writer = SummaryWriter(log_dir='logs')

for n_iter in range(100):
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)
    
img = np.zeros((3, 100, 100))
img[0] = np.arange(0, 10000).reshape(100, 100) / 10000
img[1] = 1 - np.arange(0, 10000).reshape(100, 100) / 10000

writer.add_image('my_image', img, 0)
writer.close()

# 在训练循环中记录指标
writer.add_scalar('Loss/train', loss_value, global_step)
writer.add_scalar('Accuracy/train', accuracy_value, global_step)

# 添加单个图像数据
writer.add_image('Image/train', image_tensor, global_step=None, walltime=None, dataformats='CHW')

# 可视化模型
writer.add_graph(model, input_to_model)

# 保存模型
torch.save(model.state_dict(), 'model.pth')
```

本地运行起来报错下来的数据
```shell
tensorboard --logdir=./data/logs/linear_model
```

启动后，在浏览器中输入http://127.0.0.1:6006/（Tensorboard 的默认端口为 6006），即可对刚才我们记录的数据进行可视化。

如何实现训练过程可视化呢？

In [5]:
import random
import numpy as np
import torch
from torch import nn

# 模型定义
class LinearModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.weight = nn.Parameter(torch.randn(1))
    self.bias = nn.Parameter(torch.randn(1))

  def forward(self, input):
    return (input * self.weight) + self.bias

# 数据
w = 2
b = 3
xlim = [-10, 10]
x_train = np.random.randint(low=xlim[0], high=xlim[1], size=30)
y_train = [w * x + b + random.randint(0,2) for x in x_train]

In [6]:
import os, torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

# 假设你已定义 LinearModel()
model = LinearModel()

# 建议：明确日志目录，和 TensorBoard 一致
logdir = './data/logs/linear_model'
os.makedirs(logdir, exist_ok=True)
writer = SummaryWriter(log_dir=logdir)

# 优化器（SGD+momentum+wd）
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, weight_decay=1e-2, momentum=0.9)

# 准备数据：确保 float32、shape 对齐
# x_train 是 numpy？转成 float32；y_train 形状需要和输出一致
x_t = torch.from_numpy(x_train).float()                # [N, D]
y_t = torch.tensor(y_train, dtype=torch.float32)       # [N] 或 [N,1]
# 若 model 输出是 [N,1]，而 y_t 是 [N]，需要：
if y_t.ndim == 1:
    y_t = y_t.unsqueeze(1)

criterion = nn.MSELoss()

model.train()
for n_iter in range(500):
    # （如果需要每步动态生成输入，可放回循环；但没必要每步 from_numpy 一次）
    output = model(x_t)
    loss = criterion(output, y_t)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # 写入 TensorBoard（建议写标量数值）
    writer.add_scalar('Loss/train', loss.item(), n_iter)

# 很重要：刷新并关闭，确保事件落盘
writer.flush()
writer.close()

print('TensorBoard log dir =', logdir)


# tensorboard --logdir ./Training/data/logs/linear_model --port 6006 --host 127.0.0.1


TensorBoard log dir = ./data/logs/linear_model


![image14.png](mdfiles/image14.png)